In [14]:
#from datetime import datetime
#from dateutil.relativedelta import relativedelta
#import os

import json
import pandas as pd
import requests

In [17]:
# get the headers of the taxi data
url = 'https://futar.bkk.hu/api/query/v1/ws/otp/api/where/bicycle-rental.json?key=bd29005b-80cb-499b-968b-c7be1e6b3d37&version=3&appVersion=1&includeReferences=false'

response = requests.get(url)
data = response.json()

with open('mydata.json', 'w') as file:
    json.dump(data, file, indent=4)

The structure of the data Dict:

    - "currentTime": time in Unix timestamp format (milliseconds since the Unix Epoch)
    - "version": 3,
    - "status": "OK",
    - "code": 200,
    - "text": "OK",
    - "data": all the information embedded here

The "data" key contains the following subkeys:

    - "list": the detailed information about the bikes and stations
    - "outOfRange": false    ?
    - "limitExceeded": false ?
    - "class": "listWithReferences"  ?


"list" is a list of dictionaries about bikes or bike stations

An example:
```json
        {
            "id": "42711896",
            "lat": 47.51032,
            "lon": 19.028615,
            "name": "Millen\u00e1ris",
            "code": "0213",
            "type": "stele",
            "bikes": 1,
            "spaces": 999,
            "style": {
                "icon": {
                    "name": "vehicle-rental/mol-bubi"
                }
            }
        }
```

Based on the [API documentation](https://bkkfutar.docs.apiary.io/#reference/0/bicyclerental/bicyclerental):

    - id (string):    MOL Bubi állomás azonosítója, e.g. 251962
    - lat (number):   latitude, e.g. 47.511182000000005
    - lon (number):   longitude, e.g. 19.080324700000002
    - name(string):   elnevezés, e.g. 'Dózsa György út'
    - code (string):  kód, e.g. '0612' (I guess this is the unique identitier of the station/bike)
    - type (enum):    kijelző típusa, e.g. '10inch' or 'character' (these denoted as strings)
    - bikes(number):  elérhető kerékpárok száma, e.g. 5
    - spaces (number): szabad kerékpár támaszok száma. Ha 0, akkor is elhelyezhető kerékpár az állomáson a "pót" támasznál. e.g. 10

In [10]:

# create a dataframe from the data
bicycle_rental = pd.DataFrame(data)
bicycle_rental.head(10)

,currentTime,version,status,code,text,data
class,1705653934594,3,OK,200,OK,listWithReferences
limitExceeded,1705653934594,3,OK,200,OK,False
list,1705653934594,3,OK,200,OK,"[{'id': '42711896', 'lat': 47.51032, 'lon': 19..."
outOfRange,1705653934594,3,OK,200,OK,False


A simple test calculating the average number of bikes at the stations.

In [60]:
extracteddata = []
atl = 0
for mylist in data["data"]["list"]:
    if mylist["code"] != "BIKES" and mylist["spaces"] != "0":
        extracteddata.append(mylist["bikes"])
for i in range(len(extracteddata)):
    atl += extracteddata[i]
atl /= len(extracteddata)
print(f"{str(atl)[0:4]}")
    

7.24
